## Yolo Rock Paper Scissors Notebook

### Dependencies and Imports

In [ ]:
# Install required dependencies
!pip install -q "ultralytics<=8.3.40" "supervision==0.25.1" "roboflow==1.1.54" "opencv-python==4.11.0.86"

In [ ]:
# Import necessary libraries
import os
import shutil
import git
from git import Repo, GitCommandError
import requests
from ultralytics import YOLO
from roboflow import Roboflow
from IPython.display import Image as IPyImage, display
import numpy as np

### Download the Dataset

In [ ]:

# Set home directory
HOME = os.getcwd()
print(f"Current Working Directory: {HOME}")

# Create datasets and models folders
dataset = os.path.join(HOME, "datasets")
os.makedirs(dataset, exist_ok=True)

models = os.path.join(HOME, "models")
os.makedirs(models, exist_ok=True)

# Define repository details
repo_url = "https://github.com/redhat-developer-demos/rps-game"
repo_path = os.path.join(HOME, "rps-game")
branch = "small-dataset"

# Function to clone or update the repository
def clone_or_update_repo(repo_url, repo_path, branch):
    if os.path.exists(repo_path):
        try:
            repo = Repo(repo_path)
            if repo.bare:
                print("Existing repository is bare. Removing and re-cloning...")
                shutil.rmtree(repo_path)
                raise FileNotFoundError  # Trigger the cloning process
            else:
                print(f"Repository already exists at {repo_path}. Attempting to fetch latest changes...")
                repo.git.checkout(branch)
                repo.git.pull()
                return repo_path
        except (GitCommandError, FileNotFoundError):
            print(f"Error accessing existing repository. Removing and re-cloning...")
            shutil.rmtree(repo_path)
    
    # Clone the repository
    print(f"Cloning repository from {repo_url} into {repo_path}...")
    Repo.clone_from(repo_url, repo_path, branch=branch, depth=1)
    print("Clone successful.")
    return repo_path

# Clone or update the repository
repo_path = clone_or_update_repo(repo_url, repo_path, branch)

# Define the dataset source path
data_source = os.path.join(repo_path, "roshambo-notebooks", "data")

# Copy dataset folder if it exists
if os.path.exists(data_source):
    destination_path = os.path.join(dataset, "data")
    shutil.copytree(data_source, destination_path, dirs_exist_ok=True)
    print(f"Dataset copied to {destination_path}")
else:
    print("Dataset folder not found in repository.")

# Define full dataset path
dataset_full_path = os.path.join(dataset, "data")
print(f"Full dataset path: {dataset_full_path}")


### Download PreTrained YOLO Model

In [ ]:
%cd {HOME}

# Download the Yolov11 Original Pretrained Model
dataset_location = dataset
url = "https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11s.pt"
target_dir = dataset

# Define the file path
yolov11_orig_file_path = os.path.join(target_dir, os.path.basename(url))

# Download the file
response = requests.get(url, stream=True)
if response.status_code == 200:
    with open(yolov11_orig_file_path, "wb") as file:
        for chunk in response.iter_content(chunk_size=1024):
            file.write(chunk)
    print(f"Downloaded successfully: {yolov11_orig_file_path}")
else:
    print(f"Failed to download file, status code: {response.status_code}")

### Training new YOLO Model with Rock Paper Scissors Dataset

In [ ]:
dataset_path = f"{dataset_full_path}/data.yaml"

# Load the Yolov11 Original Pretrained Model
model = YOLO(yolov11_orig_file_path)

# Number of epochs to Train
epochs = 1

# Train the model
results = model.train(
    data=dataset_path,
    epochs=epochs,
    imgsz=640,
    plots=True,
    exist_ok=True
)

### Check Model Results

In [ ]:
# List trained model results
#!ls {HOME}/runs/detect/train/

# Display training results
display(IPyImage(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600))
display(IPyImage(filename=f'{HOME}/runs/detect/train/results.png', width=600))
display(IPyImage(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600))

### Copy new Trained Model to Validate

In [ ]:
import os
import shutil

# Define paths
train_location = f"{HOME}/runs/detect/train/weights"
destination_dir = models  # Ensure dataset.location is valid

# Define the models directory inside destination_dir
models_dir = os.path.join(destination_dir, "yolo", "1")

# Define new model name based on epochs
# epochs = 1  # Ensure this variable is defined
new_model_name = f"yolo-rps.pt"

# Ensure the models/yolo directory exists
os.makedirs(models_dir, exist_ok=True)

# Copy best.pt to models/yolo with the new name
source_path = os.path.join(train_location, "best.pt")
destination_file = os.path.join(models_dir, new_model_name)

if os.path.exists(source_path):
    shutil.copy2(source_path, destination_file)
    print(f"Copied: {source_path} → {destination_file}")
else:
    print(f"Error: {source_path} not found!")

# List destination contents
print("Contents of models/yolo directory:")
os.system(f"ls {models_dir}")

### Validate new Trained RPS Model

In [ ]:
new_model_path = destination_file
#print(destination_path)

model = YOLO(new_model_path)  # load a trained model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

### Test Model with some new Images

In [ ]:
import os
import glob
import random
from IPython.display import display, Image as IPyImage

# Define test image location
images_location = f"{dataset_full_path}/test/images"
print(f"Test images directory: {images_location}")

# Select up to 30 test images
test_images = glob.glob(f"{images_location}/*.jpg")
test_images = random.sample(test_images, min(len(test_images), 30))  # Randomly pick up to 30

if not test_images:
    print("Error: No test images found.")
else:
    # Run inference on selected images
    results = model.predict(source=test_images, conf=0.25, save=True)

    # Get latest prediction folder
    predict_folders = glob.glob(f'{HOME}/runs/detect/predict*/')
    if predict_folders:
        latest_folder = max(predict_folders, key=os.path.getmtime)

        # Randomly select up to 10 images from predictions
        predicted_images = glob.glob(f'{latest_folder}/*.jpg')
        images_to_display = random.sample(predicted_images, min(len(predicted_images), 10))

        # Display selected images
        for img in images_to_display:
            display(IPyImage(filename=img, width=300))
    else:
        print("No prediction results found.")


### Export Model to ONNX

Often, when deploying computer vision models, you'll need a model format that's both flexible and compatible with multiple platforms.

Exporting YOLO11 models to ONNX format streamlines deployment and ensures optimal performance across various environments.

In [ ]:
# Define model paths using existing variables

YOLO_Rock_Paper = new_model_path
YOLO_Rock_Paper_ONNX = os.path.join(models_dir, f"yolo-rps.onnx")

# Verify if the model exists before proceeding
if not os.path.exists(YOLO_Rock_Paper):
    raise FileNotFoundError(f"Error: Model file {YOLO_Rock_Paper} not found.")

print(f"Using model: {YOLO_Rock_Paper}")

In [ ]:
# Export model to ONNX format
if not os.path.exists(YOLO_Rock_Paper_ONNX):  # Avoid re-exporting if already exists
    model.export(format="onnx")
    print(f"Exported model to ONNX format at {YOLO_Rock_Paper_ONNX}")
else:
    print(f"ONNX model already exists at {YOLO_Rock_Paper_ONNX}")

# Load ONNX model 
onnx_model = YOLO(YOLO_Rock_Paper_ONNX)

### Test Prediction with exported ONNX Model 

In [ ]:
# Define test image path dynamically
image_name = "20220216_221550_jpg.rf.02a071a383151953fcf8671fc7fca3af.jpg"
IMG = os.path.join(destination_path, "test", "images", image_name)

if not os.path.exists(IMG):
    raise FileNotFoundError(f"Error: Test image {IMG} not found.")

print(f"Using test image: {IMG}")

# Run inference
results = onnx_model(IMG)

In [ ]:
# Process results
for result in results:
    print("Processing result...")
    if hasattr(result, "boxes"):
        print("Boxes:", result.boxes)
    if hasattr(result, "masks"):
        print("Masks:", result.masks)
    if hasattr(result, "keypoints"):
        print("Keypoints:", result.keypoints)
    if hasattr(result, "probs"):
        print("Probabilities:", result.probs)
    if hasattr(result, "obb"):
        print("Oriented Boxes:", result.obb)

    # Display and optionally save results
    result.show()  # Display result
    # result.save(filename=os.path.join(destination_dir, "result.jpg"))